Much of the code was inspired by the lecture

In [1]:
import pandas as pd
import re
import nltk
from nltk.tokenize import sent_tokenize
from nltk.stem.snowball import SnowballStemmer
import numpy as np
from gensim.models import Word2Vec
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, ConfusionMatrixDisplay, confusion_matrix, roc_curve, auc, classification_report
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\orlan\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
# step 1: Exploratory Data Analysis
df1 = pd.read_csv("datasets/train.csv")
df2 = pd.read_csv("datasets/test.csv")

pos_count = 0
neg_count = 0
missing_count = 0
for idx, val in df1['Sentiment'].iteritems():
    if val == 0:
        neg_count += 1
    elif val == 1:
        pos_count += 1
    else:
        missing_count += 1
print("Training dataset")
print("positive values: " + str(pos_count))
print("negative values: " + str(neg_count))
print("missing values: " + str(missing_count))
print("total:" + str(len(df1)))
print("----------------------------")
pos_count = 0
neg_count = 0
missing_count = 0
for idx, val in df2['Sentiment'].iteritems():
    if val == 0:
        neg_count += 1
    elif val == 1:
        pos_count += 1
    else:
        missing_count += 1
print("Testing dataset")
print("positive values: " + str(pos_count))
print("negative values: " + str(neg_count))
print("missing values: " + str(missing_count))
print("total:" + str(len(df2)))


C:\Users\orlan\AppData\Local\Temp\ipykernel_16096\3867474407.py:8: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for idx, val in df1['Sentiment'].iteritems():


Training dataset
positive values: 248575
negative values: 800000
missing values: 0
total:1048575
----------------------------
Testing dataset
positive values: 182
negative values: 177
missing values: 0
total:359


C:\Users\orlan\AppData\Local\Temp\ipykernel_16096\3867474407.py:24: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for idx, val in df2['Sentiment'].iteritems():


In [3]:
# Step 2: Text Preprocessing

df1 = df1.sample(n=1000, random_state=42)
df1 = df1.reset_index(drop=True)
# df1 = pd.DataFrame({'Text': ["good movie", "not a good movie", "did not like"]}) #used for testing
# Sentence segmentation
for idx, val in df1['Text'].iteritems():
    df1.at[idx, 'Text'] = sent_tokenize(df1.at[idx, 'Text'])
    print()


# make all characters lowercase
for idx, values in df1['Text'].iteritems():
    for i in range(len(values)):
        cur = values[i]
        df1.at[idx, 'Text'][i] = cur.lower()

# Removes contractions
for idx, values in df1['Text'].iteritems():
    for i in range(len(values)):
        text = values[i]
        # text = df1.at[idx, 'Text']
        text = re.sub(r"won\'t", "will not", text)
        text = re.sub(r"can\'t", "can not", text)
        text = re.sub(r"n\'t", " not", text)
        text = re.sub(r"\'re", " are", text)
        text = re.sub(r"\'s", " is", text)
        text = re.sub(r"\'d", " would", text)
        text = re.sub(r"\'ll", " will", text)
        text = re.sub(r"\'t", " not", text)
        text = re.sub(r"\'ve", " have", text)
        text = re.sub(r"\'m", " am", text)
        df1.at[idx, 'Text'][i] = text


# Only alphabetical chars and spaces
for idx, values in df1['Text'].iteritems():
    for i in range(len(values)):
        new_val = []
        val = values[i]
        for c in val:
            if (ord(c) >= 97 and ord(c) <= 122) or c == ' ':
                new_val.append(c)
            # else:
                # new_val.append(" ") # todo revise later

        df1.at[idx, 'Text'][i] = "".join(new_val)


# stemming
stemmer = SnowballStemmer("english")
for idx, values in df1['Text'].iteritems():
    for i in range(len(values)):
        new_sentence = []
        sentence = values[i].split(" ")
        for word in sentence:
            stemmed_word = stemmer.stem(word)
            new_sentence.append(stemmed_word)

        df1.at[idx, 'Text'][i] = " ".join(new_sentence)




# reverse sentence segmentation
for idx, values in df1['Text'].iteritems():
    df1.at[idx, 'Text'] = " ".join(df1.at[idx, 'Text'])


# DF2
# Step 2: Text Preprocessing

df2 = df2.sample(n=200, random_state=42)
df2 = df2.reset_index(drop=True)

# df2 = pd.DataFrame({'Text': ["good movie", "not a good movie", "did not like"]}) #used for testing
# Sentence segmentation
for idx, val in df2['Text'].iteritems():
    df2.at[idx, 'Text'] = sent_tokenize(df2.at[idx, 'Text'])
    print()


# make all characters lowercase
for idx, values in df2['Text'].iteritems():
    for i in range(len(values)):
        cur = values[i]
        df2.at[idx, 'Text'][i] = cur.lower()

# Removes contractions
for idx, values in df2['Text'].iteritems():
    for i in range(len(values)):
        text = values[i]
        # text = df2.at[idx, 'Text']
        text = re.sub(r"won\'t", "will not", text)
        text = re.sub(r"can\'t", "can not", text)
        text = re.sub(r"n\'t", " not", text)
        text = re.sub(r"\'re", " are", text)
        text = re.sub(r"\'s", " is", text)
        text = re.sub(r"\'d", " would", text)
        text = re.sub(r"\'ll", " will", text)
        text = re.sub(r"\'t", " not", text)
        text = re.sub(r"\'ve", " have", text)
        text = re.sub(r"\'m", " am", text)
        df2.at[idx, 'Text'][i] = text


# Only alphabetical chars and spaces
for idx, values in df2['Text'].iteritems():
    for i in range(len(values)):
        new_val = []
        val = values[i]
        for c in val:
            if (ord(c) >= 97 and ord(c) <= 122) or c == ' ':
                new_val.append(c)

        df2.at[idx, 'Text'][i] = "".join(new_val)


# stemming
stemmer = SnowballStemmer("english")
for idx, values in df2['Text'].iteritems():
    for i in range(len(values)):
        new_sentence = []
        sentence = values[i].split(" ")
        for word in sentence:
            stemmed_word = stemmer.stem(word)
            new_sentence.append(stemmed_word)

        df2.at[idx, 'Text'][i] = " ".join(new_sentence) # todo might want to switch out so each cell is broken into words



# reverse sentence segmentation
for idx, values in df2['Text'].iteritems():
    df2.at[idx, 'Text'] = " ".join(df2.at[idx, 'Text'])
print("finished")

C:\Users\orlan\AppData\Local\Temp\ipykernel_16096\1804388597.py:7: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for idx, val in df1['Text'].iteritems():
C:\Users\orlan\AppData\Local\Temp\ipykernel_16096\1804388597.py:13: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for idx, values in df1['Text'].iteritems():
C:\Users\orlan\AppData\Local\Temp\ipykernel_16096\1804388597.py:19: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for idx, values in df1['Text'].iteritems():
C:\Users\orlan\AppData\Local\Temp\ipykernel_16096\1804388597.py:37: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for idx, values in df1['Text'].iteritems():
C:\Users\orlan\AppData\Local\Temp\ipykernel_16096\1804388597.py:52: FutureWarning: iteritems is deprecated and will be removed in a future version. 









































































































































































































finished


C:\Users\orlan\AppData\Local\Temp\ipykernel_16096\1804388597.py:66: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for idx, values in df1['Text'].iteritems():
C:\Users\orlan\AppData\Local\Temp\ipykernel_16096\1804388597.py:78: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for idx, val in df2['Text'].iteritems():
C:\Users\orlan\AppData\Local\Temp\ipykernel_16096\1804388597.py:84: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for idx, values in df2['Text'].iteritems():
C:\Users\orlan\AppData\Local\Temp\ipykernel_16096\1804388597.py:90: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for idx, values in df2['Text'].iteritems():
C:\Users\orlan\AppData\Local\Temp\ipykernel_16096\1804388597.py:108: FutureWarning: iteritems is deprecated and will be removed in a future version

In [4]:
# Bag of Words
# calculates the number of times each word appears 
word2count = {}
for idx, sentence in df1['Text'].iteritems():
    words = nltk.word_tokenize(sentence)
    for word in words:
        if word not in word2count.keys():
            word2count[word] = 1
        else:
            word2count[word] += 1

unique_words = list(word2count.keys())

bag_of_words = []
# fills in bag of words matrix
for idx, sentence in df1['Text'].iteritems():
    words = nltk.word_tokenize(sentence) # breaks sentence down into words
    bag_vector = np.zeros(len(unique_words))
    for w in words:
        for index, word in enumerate(unique_words):
            if word == w:
                bag_vector[index] += 1
    bag_of_words.append(bag_vector)

x_train = np.array(bag_of_words)
y_train = df1["Sentiment"]

bag_of_words_test = []

for sentence in df2['Text']:
    words = nltk.word_tokenize(sentence)
    bag_vector = np.zeros(len(unique_words))
    for w in words:
        for index, word in enumerate(unique_words):
            if word == w:
                bag_vector[index] += 1
    bag_of_words_test.append(bag_vector)

x_test = np.array(bag_of_words_test)
y_test = df2["Sentiment"]



C:\Users\orlan\AppData\Local\Temp\ipykernel_16096\2395915059.py:4: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for idx, sentence in df1['Text'].iteritems():
C:\Users\orlan\AppData\Local\Temp\ipykernel_16096\2395915059.py:16: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for idx, sentence in df1['Text'].iteritems():


In [5]:
lc = LogisticRegression()
svc = SVC(probability=True)
nbc = GaussianNB()
rfc = RandomForestClassifier()

In [6]:
lc.fit(x_train, y_train)
svc.fit(x_train, y_train)
nbc.fit(x_train, y_train)
rfc.fit(x_train, y_train)

RandomForestClassifier()

In [7]:
y_lc_predicted = lc.predict(x_test)
y_lc_pred_proba = lc.predict_proba(x_test)

y_svc_predicted = svc.predict(x_test)
y_svc_pred_proba = svc.predict_proba(x_test)

y_nbc_predicted = nbc.predict(x_test)
y_nbc_pred_proba = nbc.predict_proba(x_test)

y_rfc_predicted = rfc.predict(x_test)
y_rfc_pred_proba = rfc.predict_proba(x_test)

In [8]:
print(classification_report(y_test, y_lc_predicted))
print(classification_report(y_test, y_svc_predicted))
print(classification_report(y_test, y_nbc_predicted))
print(classification_report(y_test, y_rfc_predicted))

              precision    recall  f1-score   support

           0       0.60      0.93      0.73       107
           1       0.78      0.30      0.43        93

    accuracy                           0.64       200
   macro avg       0.69      0.61      0.58       200
weighted avg       0.68      0.64      0.59       200

              precision    recall  f1-score   support

           0       0.54      1.00      0.70       107
           1       1.00      0.02      0.04        93

    accuracy                           0.55       200
   macro avg       0.77      0.51      0.37       200
weighted avg       0.75      0.55      0.39       200

              precision    recall  f1-score   support

           0       0.57      0.77      0.65       107
           1       0.55      0.33      0.42        93

    accuracy                           0.56       200
   macro avg       0.56      0.55      0.53       200
weighted avg       0.56      0.56      0.54       200

              preci

In [9]:
tr_idf_model = TfidfVectorizer()
tf_idf_vector = tr_idf_model.fit_transform(df1["Text"]) # vectorizes text column
tf_idf_vector = tf_idf_vector.toarray()

# Use the same vectorizer to transform the test data
tf_idf_vector_test = tr_idf_model.transform(df2["Text"])
tf_idf_vector_test = tf_idf_vector_test.toarray()

In [10]:
x_train = tf_idf_vector
y_train = df1["Sentiment"]

x_test = tf_idf_vector_test
y_test = df2["Sentiment"]

In [11]:
lc = LogisticRegression()
svc = SVC(probability=True)
nbc = GaussianNB()
rfc = RandomForestClassifier()

In [12]:
lc.fit(x_train, y_train)
svc.fit(x_train, y_train)
nbc.fit(x_train, y_train)
rfc.fit(x_train, y_train)

RandomForestClassifier()

In [13]:
y_lc_predicted = lc.predict(x_test)
y_lc_pred_proba = lc.predict_proba(x_test)

y_svc_predicted = svc.predict(x_test)
y_svc_pred_proba = svc.predict_proba(x_test)

y_nbc_predicted = nbc.predict(x_test)
y_nbc_pred_proba = nbc.predict_proba(x_test)

y_rfc_predicted = rfc.predict(x_test)
y_rfc_pred_proba = rfc.predict_proba(x_test)

In [14]:
print(classification_report(y_test, y_lc_predicted))
print(classification_report(y_test, y_svc_predicted))
print(classification_report(y_test, y_nbc_predicted))
print(classification_report(y_test, y_rfc_predicted))

              precision    recall  f1-score   support

           0       0.54      1.00      0.70       107
           1       1.00      0.01      0.02        93

    accuracy                           0.54       200
   macro avg       0.77      0.51      0.36       200
weighted avg       0.75      0.54      0.38       200

              precision    recall  f1-score   support

           0       0.54      1.00      0.70       107
           1       1.00      0.01      0.02        93

    accuracy                           0.54       200
   macro avg       0.77      0.51      0.36       200
weighted avg       0.75      0.54      0.38       200

              precision    recall  f1-score   support

           0       0.57      0.75      0.65       107
           1       0.54      0.34      0.42        93

    accuracy                           0.56       200
   macro avg       0.55      0.55      0.53       200
weighted avg       0.56      0.56      0.54       200

              preci

In [15]:
# word2vec, code inspired by https://spotintelligence.com/2023/02/15/word2vec-for-text-classification/
x_train = [str(sentence).split() for sentence in df1['Text']]
x_test = [str(sentence).split() for sentence in df2['Text']]
y_train = list(df1["Sentiment"])
y_test = list(df2["Sentiment"])
w2v_model = Word2Vec(x_train, window=5, min_count=5, workers=4)

def vectorize(sentence):
    words = sentence
    words_vecs = [w2v_model.wv[word] for word in words if word in w2v_model.wv]
    if len(words_vecs) == 0:
        return np.zeros(100)
    words_vecs = np.array(words_vecs)
    return words_vecs.mean(axis=0)

x_train = np.array([vectorize(sentence) for sentence in x_train])
x_test = np.array([vectorize(sentence) for sentence in x_test])




In [16]:
lc = LogisticRegression()
svc = SVC(probability=True)
nbc = GaussianNB()
rfc = RandomForestClassifier()

In [17]:
lc.fit(x_train, y_train)
svc.fit(x_train, y_train)
nbc.fit(x_train, y_train)
rfc.fit(x_train, y_train)

RandomForestClassifier()

In [18]:
y_lc_predicted = lc.predict(x_test)
y_lc_pred_proba = lc.predict_proba(x_test)

y_svc_predicted = svc.predict(x_test)
y_svc_pred_proba = svc.predict_proba(x_test)

y_nbc_predicted = nbc.predict(x_test)
y_nbc_pred_proba = nbc.predict_proba(x_test)

y_rfc_predicted = rfc.predict(x_test)
y_rfc_pred_proba = rfc.predict_proba(x_test)

In [19]:
print(classification_report(y_test, y_lc_predicted))
print(classification_report(y_test, y_svc_predicted))
print(classification_report(y_test, y_nbc_predicted))
print(classification_report(y_test, y_rfc_predicted))

              precision    recall  f1-score   support

           0       0.54      1.00      0.70       107
           1       0.00      0.00      0.00        93

    accuracy                           0.54       200
   macro avg       0.27      0.50      0.35       200
weighted avg       0.29      0.54      0.37       200

              precision    recall  f1-score   support

           0       0.54      1.00      0.70       107
           1       0.00      0.00      0.00        93

    accuracy                           0.54       200
   macro avg       0.27      0.50      0.35       200
weighted avg       0.29      0.54      0.37       200

              precision    recall  f1-score   support

           0       0.54      0.70      0.61       107
           1       0.48      0.31      0.38        93

    accuracy                           0.52       200
   macro avg       0.51      0.51      0.49       200
weighted avg       0.51      0.52      0.50       200

              preci

C:\Users\orlan\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\orlan\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\orlan\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average,

MODEL EVALUATION: 
All models performed quite similarly. however it seems like bag-of-words performed with the highest accuracy followed by tf*idf, and then word2vector. For bag-of-words, the logistic regression model performed the best, for the tf*idf, the random forest classifier performed the best, and for the word2vector, the support vector machine and logistic regression model tied for most accurate. The model with the highest accuracy was the logisitc regression model for the bag_of_words which was only 64 percent accurate. This could definitely be improve, particularly in the preprocessing stage. If lemmatization was used, there could be a notable increase in accuracy. Moreover, if I used more datapoints, the might also be an increase in accuracy. Lastly, there might be some inaccuracies in my code which could be hindering the results as well.  